Models Assessment

In [2]:
import pandas as pd
import numpy as np

#model imports
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier,XGBRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

#metrics
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from statistics import mean

#useful functions
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE


In [3]:
#Define an instance of the models
#We are using the best 
DT = DecisionTreeClassifier()
RF = RandomForestClassifier(verbose = 1, n_jobs=-1)
XGB = XGBClassifier()
KNN = KNeighborsClassifier(n_neighbors = 50)
MLP = MLPClassifier(activation='relu',
    solver='sgd',learning_rate='invscaling',
    learning_rate_init=0.001,
    batch_size=100,verbose = True)

#Creating an instance of our encoder for the target
le = LabelEncoder()

In [4]:
# Load data
data= pd.read_csv("train_data_enriched.csv", index_col="Claim Identifier")
data_test = pd.read_csv("test_data_enriched.csv",index_col="Claim Identifier")

C:\Users\gonca\AppData\Local\Temp\ipykernel_8476\1043285265.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv("train_data_enriched.csv", index_col="Claim Identifier")


In [5]:
def impute_missing_values(X_train, X_val, target_column, algorithm,validation = True):

    # Separating the missing values from the non missing values
    available_data = X_train[X_train[target_column].notna()]
    missing_X_train = X_train[X_train[target_column].isna()]
    missing_X_val = X_val[X_val[target_column].isna()]

    # Making sure if there is enough data for inputing, returning it if not
    if len(missing_X_train) == 0:
        print(f"no missing values to input on {target_column}")
        return X_train, X_val

    # Separating the target column from the rest
    X_available = available_data.drop(columns=[target_column])
    y_available = available_data[target_column]

    # Making sure our columns are consistent
    X_available = X_available.select_dtypes(include=["number"])
    missing_X_train = missing_X_train.select_dtypes(include=["number"])
    missing_X_val = missing_X_val.select_dtypes(include=["number"])

    common_columns = X_available.columns.intersection(missing_X_train.columns).intersection(missing_X_val.columns)
    X_available = X_available[common_columns]
    missing_X_train = missing_X_train[common_columns]
    missing_X_val = missing_X_val[common_columns]

    # Making sure there is any column after keeping the common columns
    if X_available.shape[1] == 0:
        print(f"Without any column to input in {target_column}")
        return X_train, X_val

    # Training the model with the available data
    model = algorithm
    model.fit(X_available, y_available)

    # Prediting the missing values
    predicted_train = model.predict(missing_X_train)
    if validation:
        predicted_val = model.predict(missing_X_val)

    # Filling the training and validation with predictions. The latter is only filled if the argument is true
    X_train = X_train.copy()
    X_train.loc[X_train[target_column].isna(), target_column] = predicted_train

    if validation:
        X_val = X_val.copy()
        X_val.loc[X_val[target_column].isna(), target_column] = predicted_val

    return X_train, X_val

In [6]:
#This helper fuction was used for finding problems with later functions
'''def check_missing_values(data, step_name):
    print(f"\n{step_name}: Valores ausentes restantes:")
    print(data.isnull().sum()[data.isnull().sum() > 0])'''

'def check_missing_values(data, step_name):\n    print(f"\n{step_name}: Valores ausentes restantes:")\n    print(data.isnull().sum()[data.isnull().sum() > 0])'

In [7]:
def handle_outliers(data, column):
    # Handles outliers in a numerical column by replacing values outside the interquartile range (IQR) with missing values

    # Makes sure we only treat outliers in columns that have any data
    if data[column].notnull().sum() > 0: 

        # Calculating inter quantile range limits
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Changing the ouliers to missing values
        data[column] = np.where(data[column] < lower_bound, np.nan, data[column])
        data[column] = np.where(data[column] > upper_bound, np.nan, data[column])

    return data

In [8]:
def scale_numerical(column, X_train, X_val, scaler):
    
    # Make sure the column is numerical
    if not pd.api.types.is_numeric_dtype(X_train[column]):
        print(f"Columm '{column}' is not numerical and will be ignored")
        return

    # Scaling the data
    try:
        X_train[column] = scaler.fit_transform(X_train[[column]])
        X_val[column] = scaler.transform(X_val[[column]])
    except ValueError as e:
        print(f"Mistake scaling the column '{column}': {e}")

In [9]:
def  claim_carrier_categories(X_train, X_val):
    
    # Define a function to categorize each carrier based on its claim count
    count = X_train['Carrier Name'].value_counts()
    def categorize_claims(count):
        if count >= 40000:
            return 2
        elif 4000 <= count < 40000:
            return 1
        else:
            return 0

    # Apply the categorization to create a mapping dictionary
    carrier_category_map = count.apply(categorize_claims)

    # Map the `Carrier Name` to the new `Carrier Claim Category`
    X_train['Carrier Claim Category'] = X_train['Carrier Name'].map(carrier_category_map)
    X_val['Carrier Claim Category'] = X_val['Carrier Name'].map(carrier_category_map)

    return X_train.drop(["Carrier Name"], axis = 1, inplace = True) , X_val.drop(["Carrier Name"], axis = 1, inplace = True)


In [10]:
# Categorical encoder function
def categorical_prop_encode(X_train, X_val, feature):
    proportion = X_train[feature].value_counts(normalize = True)  # Get the porportion of each category
    X_train[feature] = X_train[feature].map(proportion)  # Map the porportions in the column
    X_val[feature] = X_val[feature].map(proportion) # Do the same for the validation subset
    X_val[feature] = X_val[feature].fillna(0)  # Handle categories in X_val not seen in X_train with 0


In [11]:
def Rfe(algorithm,X_train,y_train,X_val,y_val):

    #Generating the variables where we will store our results
    nof_list = np.arange(1, len(X_train.columns) + 1)            
    high_score = 0
    opt_n_features = 0
    train_score_list = []
    val_score_list = []

    #Variable where we will store the optimum amount of features
    best_rfe = None

    model = algorithm

    for n in nof_list:
        rfe = RFE(estimator=model, n_features_to_select=n)
    
    # Fitting the model to rfe
        X_train_rfe = rfe.fit_transform(X_train, y_train)
        X_val_rfe = rfe.transform(X_val)
    
    # Training and predicting
        model.fit(X_train_rfe, y_train)
        pred_train = model.predict(X_train_rfe)
        pred_val = model.predict(X_val_rfe)
    
    # Evaluating using the macro f1_score
        train_score = f1_score(y_train, pred_train, average="macro")
        val_score = f1_score(y_val, pred_val, average="macro")
        train_score_list.append(train_score)
        val_score_list.append(val_score)
    
    # Checking if this is the best combination of features so far
        if val_score >= high_score:
            high_score = val_score
            opt_n_features = n
            best_rfe = rfe  # Storing the rfe with the best number of features

# Checking what amount of features and which features where the best for the model
    selected_features = X_train.columns[best_rfe.support_].tolist()

    print("Optimal number of features: %d" % opt_n_features)
    print("Score with %d features: %f" % (opt_n_features, high_score))
    print("Selected Features:\n", selected_features)

    return selected_features


In [12]:
def cv_scores(X, y, num_features, cat_features, num_imputing_algorithm=XGBRegressor(), cat_imputing_algorithm=XGBClassifier(), scaling_outlier= False, scaler=MinMaxScaler(), rfe = False):
    """
    Takes as argument the predictors and the target, the models used for imputing numerical and categorical 
    features, if any scaling and outlier removal should be performed,what scaling method should be used and if feature selection with rfe should be used.
    Then it returns the results obtained from the stratified cross-validation for the given models.
    """
    skf = StratifiedKFold(n_splits=5)

    # Generating the lists to store our results
    precision_scores_train = [[],[],[],[],[]]
    precision_scores_val = [[],[],[],[],[]]  
    recall_scores_train = [[],[],[],[],[]]
    recall_scores_val = [[],[],[],[],[]]
    f1_scores_train =  [[],[],[],[],[]]
    f1_scores_val =  [[],[],[],[],[]]

    precision_scores_train_mean = []
    precision_scores_val_mean = [] 
    recall_scores_train_mean = []
    recall_scores_val_mean = []
    f1_scores_train_mean =  []
    f1_scores_val_mean =  []
    
    for train_index, test_index in skf.split(X, y):
        # Dividing our data in validation and train
        X_train, X_val = X.iloc[train_index].copy(), X.iloc[test_index].copy()
        y_train, y_val = y.iloc[train_index].copy(), y.iloc[test_index].copy()

        # Filling missing values
        for column in num_features:
            X_train, X_val = impute_missing_values(X_train,X_val, column, num_imputing_algorithm)
        
        # Removing inconsistencies on the train
        inconsistent = X_train[(X_train['Age at Injury'] > 80) | (X_train["Age at Injury"] < 16)].index
        X_train = X_train.loc[~X_train.index.isin(inconsistent)]
        y_train = y_train.loc[~y_train.index.isin(inconsistent)]

        # Performing scaling and outlier treatment dependent on the boolean
        if scaling_outlier:
            for column in num_features:
                handle_outliers(X_train, column)
                X_train, X_val = impute_missing_values(X_train,X_val, column, num_imputing_algorithm,validation= False)

            for column in num_features:
                scale_numerical(column, X_train, X_val, scaler)
                
        # Creating an ordinal variable
        claim_carrier_categories(X_train, X_val)

        #Filling missing values in the ordinal variable that might appear on X_val
        X_val, X_train = impute_missing_values(X_val,X_train, "Carrier Claim Category", cat_imputing_algorithm, validation=False)

        # Categorical Prop Encoding
        for cat_feature in cat_features:
            categorical_prop_encode(X_train, X_val, cat_feature)
            if scaling_outlier:
                scale_numerical("Carrier Claim Category", X_train, X_val, scaler)

        # Selecting features with Rfe
        if rfe:
            Selected_features = Rfe(XGBClassifier(), X_train, y_train, X_val, y_val)
            X_train = X_train[Selected_features]
            X_val = X_val[Selected_features]

        # Training the classification models
        DT.fit(X_train, y_train)
        print("Done DT")
        RF.fit(X_train, y_train)
        print("Done RF")
        XGB.fit(X_train, y_train)
        print("Done XGB")
        KNN.fit(X_train, y_train)
        print("Done KNN")
        MLP.fit(X_train, y_train)
        print("Done MLP")

        # Making the predictions for the training and validation data
        pred_train_DT = DT.predict(X_train)
        pred_train_RF = RF.predict(X_train)
        pred_train_XGB = XGB.predict(X_train)
        pred_train_KNN = KNN.predict(X_train)
        pred_train_MLP = MLP.predict(X_train)
        print("Done training predictions")
        
        pred_val_DT = DT.predict(X_val)
        pred_val_RF = RF.predict(X_val)
        pred_val_XGB = XGB.predict(X_val)
        pred_val_KNN = KNN.predict(X_val)
        pred_val_MLP = MLP.predict(X_val)
        print("Done validation predictions")

        # Calculating and storing the scores
        i = 0
        for predictions in [pred_train_DT,pred_train_RF,pred_train_XGB,pred_train_KNN,pred_train_MLP]:
            precision_scores_train[i].append(precision_score(y_train, predictions, average='macro'))
            recall_scores_train[i].append(recall_score(y_train, predictions, average='macro'))
            f1_scores_train[i].append(f1_score(y_train, predictions, average='macro'))
            i+=1
        j=0
        for predictions in [pred_val_DT,pred_val_RF,pred_val_XGB,pred_val_KNN,pred_val_MLP]:
            precision_scores_val[j].append(precision_score(y_val, predictions, average='macro'))
            recall_scores_val[j].append(recall_score(y_val, predictions, average='macro'))
            f1_scores_val[j].append(f1_score(y_val, predictions, average='macro'))
            j+=1

        # Check the confusion matrixes of our predictions
        print(confusion_matrix(y_val, pred_val_DT))
        print(confusion_matrix(y_val, pred_val_RF))
        print(confusion_matrix(y_val, pred_val_XGB))
        print(confusion_matrix(y_val, pred_val_KNN))
        print(confusion_matrix(y_val, pred_val_MLP))

    # Aggregating the average results across the folds
    for l in range(0,5): 
        precision_scores_train_mean.append(mean(precision_scores_train[l]))
        precision_scores_val_mean.append(mean(precision_scores_val[l]))
        recall_scores_train_mean.append(mean(recall_scores_train[l]))
        recall_scores_val_mean.append(mean(recall_scores_val[l]))
        f1_scores_train_mean.append(mean(f1_scores_train[l]))
        f1_scores_val_mean.append(mean(f1_scores_val[l]))

    # Storing the results in a dataframe
    model_results = pd.DataFrame(data={
        'Train_precision': precision_scores_train_mean,
        'Test_precision': precision_scores_val_mean,
        'Train_recall': recall_scores_train_mean,
        'Test_recall': recall_scores_val_mean,
        'Train_f1_score': f1_scores_train_mean,
        'Test_f1_score': f1_scores_val_mean,
    }, index=["Decision Tree","Random Forest","XGBoost", "KNearestNeighbors","Multi Layer Perceptron"])

    return model_results

In [13]:
def test_prediction(model, X, y, num_features, cat_features, data_test, 
                    num_imputing_algorithm=XGBRegressor(), 
                    cat_imputing_algorithm=XGBClassifier(), scaling_outlier = False , 
                    scaler=MinMaxScaler()):

    # Impute missing values
    for column in num_features:
        X, data_test = impute_missing_values(X,data_test,column, num_imputing_algorithm)

    # Remove inconsistencies
    inconsistent = X[(X['Age at Injury'] > 80) | (X["Age at Injury"] < 16)].index
    X.drop(inconsistent, inplace=True)
    y.drop(inconsistent, inplace=True)

    # Scale and remove outliers if specified
    if scaling_outlier:
        for column in num_features:
            handle_outliers(X, column)
            X, data_test = impute_missing_values(X,data_test, column, num_imputing_algorithm,validation = False)

        for column in num_features:
            scale_numerical(column, X, data_test, scaler)
        
    # Creating an ordinal variable
    claim_carrier_categories(X, data_test)
    
    # Inputing missing values that might appear on data_test in the new variable
    data_test, X = impute_missing_values(data_test, X, "Carrier Claim Category", cat_imputing_algorithm,validation= False)

    # Categorical Prop Encoding
    for cat_feature in cat_features:
        categorical_prop_encode(X, data_test, cat_feature)
        if scaling_outlier:
            scale_numerical("Carrier Claim Category", X, data_test, scaler)

    # Fitting the model, making the predictions and reverting the claim injury types back to their string form
    model.fit(X, y)
    pred_test = model.predict(data_test)
    pred_test = le.inverse_transform(pred_test)

    # Saving the final submission dataframe with indexes of data_test
    submission_df = pd.DataFrame({
        "Claim Injury Type": pred_test
    }, index=data_test.index)
    
    return submission_df

In [14]:
#Label enconding our target variable 
data["Claim Injury Type"] = le.fit_transform(data["Claim Injury Type"])

In [15]:
'''Dropping redundant variables that carry almost the same information (are extremely correlated (|0.8|))
We believe it was better to keep Age at Injury than birth year since it should be more related to the injury claim type (it will be tested later)
The same logic was applied to dropping the other two dates and two DSA variables since we believe Accident date to be more important'''

data = data.loc[:, ~data.columns.isin(['Birth Year', 'Assembly Date', 'C-2 Date', 'Assembly Date DSA', 'First Hearing Date DSA'])]
data_test = data_test.loc[:, ~data_test.columns.isin(['Birth Year', 'Assembly Date', 'C-2 Date', 'Assembly Date DSA', 'First Hearing Date DSA'])]


In [16]:
'''Since the codes always seem to provide the same or more information than the descriptions (have more categories),
and the codes are consistent (always only having 1 description for code, while descriptions may have multiple codes)
And Crámer's V says they have a very high association
we will drop the description columns.'''
data.drop(['Industry Code Description','WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description'], axis = 1,inplace = True)
data_test.drop(['Industry Code Description','WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description'], axis = 1,inplace = True)


In [17]:
'''Dropping redundant variables that carry almost the same information (have an association above or equal to 0.8)
We chose to keep County of Injury above Zip Code and District Name (these 3 have a high association) because it is the easist to interpret and the one we looked more in detail in the exploratory analysis
We kept the new variable we made called body section because it keeps most of the same information of the body part code but with a much lower cardinality
Lastly we only remove the variable Carrier Name in the function where we create the new variable with lower cardinality because it is need to create that new variable'''
#"WCIO Part Of Body Code","District Name"
data.drop(['Zip Code',"WCIO Part Of Body Code","District Name"], axis=1, inplace = True)
data_test.drop(["Zip Code","WCIO Part Of Body Code","District Name"], axis=1 , inplace = True)

In [18]:
num_features = ['Age at Injury', 'Average Weekly Wage', 'IME-4 Count', 'Number of Dependents',
                "Accident Year","Accident Month","Accident Day","Accident DayOfWeek",
                "C-2 Date DSA","C-3 Date DSA" ,"Accident Year","Accident Month",
                "Accident Day","Accident DayOfWeek","Accident Date","C-3 Date","First Hearing Date"]

In [19]:
cat_features = [
    "Alternative Dispute Resolution",
    "Carrier Type",
    "County of Injury",
    "Gender",
    "Industry Code",
    "Medical Fee Region",
    "WCIO Cause of Injury Code",
    "WCIO Nature of Injury Code",
    "Age at Injury Category",
    "Carrier Claim Category",
    "Body Section",
]

In [20]:
#We remove the missing values in this variable since they are only 5 rows and would never make a big difference in the 600 000+ rows of the dataset
data.dropna(axis= 0, inplace = True, subset="Alternative Dispute Resolution")

In [21]:
X = data.drop(["Claim Injury Type","Agreement Reached"], axis = 1)

In [22]:
y = data["Claim Injury Type"]

In [23]:
y2 = data["Agreement Reached"]

In [24]:
results = cv_scores(X, y,num_features,cat_features,scaling_outlier = True)

no missing values to input on Number of Dependents
no missing values to input on Accident Year
no missing values to input on Accident Month
no missing values to input on Accident Day
no missing values to input on Accident DayOfWeek
no missing values to input on Age at Injury
no missing values to input on Number of Dependents
no missing values to input on Accident Month
no missing values to input on Accident DayOfWeek
no missing values to input on Accident Year
no missing values to input on Accident Month
no missing values to input on Accident Day
no missing values to input on Accident DayOfWeek
Done DT


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   48.1s finished


Done RF
Done XGB
Done KNN
Iteration 1, loss = 1.05579236
Iteration 2, loss = 1.00605287
Iteration 3, loss = 1.00593230
Iteration 4, loss = 1.00586784
Iteration 5, loss = 1.00582081
Iteration 6, loss = 1.00578257
Iteration 7, loss = 1.00574961
Iteration 8, loss = 1.00572040
Iteration 9, loss = 1.00569374
Iteration 10, loss = 1.00566915
Iteration 11, loss = 1.00564611
Iteration 12, loss = 1.00562450
Iteration 13, loss = 1.00560400
Iteration 14, loss = 1.00558443
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Done MLP


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    1.7s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    5.5s finished


Done training predictions


...

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.4s finished


Done validation predictions


C:\Users\gonca\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gonca\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gonca\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gonca\anaconda3\Lib\site-packag

[[  918   632   745   177    21     0     0     2]
 [  652 11615 41974  3696   263     7     0     7]
 [   70  1987 10202  1363   150     6     0     3]
 [  152  4097 20973  3970   484    19     0     7]
 [   80  1358  4517  3081   601    17     0     2]
 [    8    59   318   381    71     3     0     2]
 [    0     0     8     9     3     0     0     0]
 [    3    11    31    43     5     0     0     1]]
[[ 1087   571   759    71     7     0     0     0]
 [  152  7572 50024   413    53     0     0     0]
 [    3   444 13076   233    25     0     0     0]
 [    4  1185 27245  1242    26     0     0     0]
 [    1   392  5222  3588   453     0     0     0]
 [    0    11   219   584    28     0     0     0]
 [    0     1     3    14     2     0     0     0]
 [    0    10    35    49     0     0     0     0]]
[[ 1087   429   932    37    10     0     0     0]
 [  218 11613 46052   264    66     0     0     1]
 [    2   836 12721   171    51     0     0     0]
 [    8  1416 27235   945    

C:\Users\gonca\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gonca\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [48]:
#Check the results
results

,Train_precision,Test_precision,Train_recall,Test_recall,Train_f1_score,Test_f1_score
Decision Tree,0.999950,0.213073,0.999990,0.224442,0.999970,0.196063
Random Forest,0.999973,0.316299,0.999958,0.267772,0.999966,0.252458
XGBoost,0.795255,0.291958,0.580957,0.259127,0.623074,0.239476
KNearestNeighbors,0.381528,0.311876,0.302539,0.268804,0.314318,0.272512
Multi Layer Perceptron,0.286770,0.243040,0.203047,0.202977,0.188041,0.188804


In [179]:
# Choose best model from KFold of cross validation
# Train model on whole train and predict test data
submission = test_prediction(XGBClassifier(),X,y,num_features,cat_features,data_test)
submission

no missing values to input on Number of Dependents
no missing values to input on Accident Year
no missing values to input on Accident Month
no missing values to input on Accident Day
no missing values to input on Accident DayOfWeek
no missing values to input on Alternative Dispute Resolution


C:\Users\gonca\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:03:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:52: Empty dataset at worker: 0
  warnings.warn(smsg, UserWarning)


,Claim Injury Type
Claim Identifier,
6165911,2. NON-COMP
6166141,2. NON-COMP
6165907,2. NON-COMP
6166047,2. NON-COMP
6166102,2. NON-COMP
...,...
6553137,3. MED ONLY
6553119,4. TEMPORARY
6553542,4. TEMPORARY


In [181]:
#export to csv
submission.to_csv("Submission.csv")